In [45]:
import torch

from semantic_memory import vsm
from tqdm import trange

In [15]:
embeddings = []
with open("spose_embedding_66d_sorted.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        vector = [float(x) for x in line.split()]
        embeddings.append(vector)

embeddings = torch.tensor(embeddings)

In [20]:
vocab = open("unique_id.txt", "r").readlines()
vocab = [x.strip() for x in vocab]

In [26]:
things = vsm.VectorSpaceModel("THINGS 66d")
things.load_vectors_from_tensor(embeddings, vocab)

In [28]:
things.neighbor("mouse1",k=10)

[('rat', 0.98600834608078),
 ('warthog', 0.9813143014907837),
 ('chipmunk', 0.980313241481781),
 ('rhinoceros', 0.9765945076942444),
 ('bear', 0.9756324291229248),
 ('fox', 0.9725315570831299),
 ('chinchilla', 0.9714034795761108),
 ('cougar', 0.9698286652565002),
 ('coyote', 0.9682605266571045),
 ('mongoose', 0.9668706655502319)]

In [29]:
def dot_product_similarity(v1, v2):
    return torch.dot(v1, v2)

In [33]:
dot_product_similarity(things("mouse1").squeeze(0), things("rat").squeeze(0))

tensor(7.5295)

In [35]:
things.neighbor("mouse1", k = len(vocab), )

[('rat', 0.98600834608078),
 ('warthog', 0.9813143014907837),
 ('chipmunk', 0.980313241481781),
 ('rhinoceros', 0.9765945076942444),
 ('bear', 0.9756324291229248),
 ('fox', 0.9725315570831299),
 ('chinchilla', 0.9714034795761108),
 ('cougar', 0.9698286652565002),
 ('coyote', 0.9682605266571045),
 ('mongoose', 0.9668706655502319),
 ('koala', 0.965898871421814),
 ('gazelle', 0.9658738374710083),
 ('boar', 0.964140772819519),
 ('antelope', 0.9639860391616821),
 ('tiger', 0.9625461101531982),
 ('hyena', 0.9601081013679504),
 ('mole', 0.9587732553482056),
 ('elephant', 0.958243191242218),
 ('kangaroo', 0.9576485753059387),
 ('meerkat', 0.95573490858078),
 ('possum', 0.9542941451072693),
 ('aardvark', 0.9531489014625549),
 ('squirrel', 0.9523568153381348),
 ('cat', 0.9521333575248718),
 ('raccoon', 0.9515522122383118),
 ('hedgehog', 0.9508214592933655),
 ('guinea_pig', 0.9500805735588074),
 ('beaver', 0.9499233365058899),
 ('otter', 0.9494654536247253),
 ('hippopotamus', 0.9453458189964294),

In [46]:
def embedding2sim(embedding):
    # Convert embedding to similarity matrix
    n_objects = embedding.size(0)

    # Compute the similarity matrix
    sim = torch.mm(embedding, embedding.T)
    esim = torch.exp(sim)

    print("Initialized similarities")

    # Initialize the cp matrix
    cp = torch.zeros(n_objects, n_objects)

    print("Initialized matrix")

    # Compute the similarity matrix
    for i in range(n_objects):
        for j in trange(i + 1, n_objects):
            ctmp = torch.zeros(n_objects)
            for k in range(n_objects):
                if k == i or k == j:
                    continue
                ctmp[k] = esim[i, j] / (esim[i, j] + esim[i, k] + esim[j, k])
            cp[i, j] = torch.sum(ctmp)

    # Normalize the cp matrix
    cp = cp / (n_objects - 2)
    cp = cp + cp.T  # Make the matrix symmetric
    cp[torch.eye(n_objects).bool()] = 1  # Set the diagonal to 1

    return cp

In [47]:
spose_sim = embedding2sim(embeddings)

Initialized similarities
Initialized matrix


 20%|█▉        | 365/1852 [00:05<00:22, 65.79it/s]


KeyboardInterrupt: 

In [49]:
n_objects = embeddings.size(0)
c=0
for i in range(n_objects):
    for j in range(i + 1, n_objects):
        c+=1
        # ctmp = torch.zeros(n_objects)
        # for k in range(n_objects):

In [50]:
c

1717731